# **DSFM Project**: Predict House Prices

Creator: [Data Science for Managers - EPFL Program](https://www.dsfm.ch)  
Source:  [https://github.com/dsfm-org/code-bank.git](https://github.com/dsfm-org/code-bank.git)  
License: [MIT License](https://opensource.org/licenses/MIT). See open source [license](LICENSE) in the Code Bank repository. 

-------------

## Overview

In this project, we will predict the sale price of houses located in Ames, Iowa, based on sevaral features. This is a regression problem as we are predicting a continuous outcome.

<img src="https://storage.needpix.com/rsynced_images/new-home-for-sale-1405784329d8m.jpg" width="500" height="500" align="center"/>

Image source: https://storage.needpix.com/rsynced_images/new-home-for-sale-1405784329d8m.jpg

### The Housing Dataset 

The dataset consists of 1460 houses with 80 features, described in the *data_description.txt* file in this directory. The target feature is the *SalesPrice* in US Dollar. As you will see, this dataset includes numerous features of different kinds: a good playground for feature engineering and a good lesson for carefully understanding the dataset. Furthermore, this project shows the power of tree-based models, such as random forests and gradient boosting trees.

Dataset source: [Kaggle Ames Housing Dataset challenge](https://www.kaggle.com/prevek18/ames-housing-dataset)

To assess the quality of your predictions, consider the root mean square error (RMSE) of the logarithm of the predicted value and the logarithm of the observed *SalesPrice*.

The problem is divided into several parts. For each part, you will have time to work on the question yourself. Feel free to go back to the Demo, use Google/Stackoverflow and work with your neighbour. Together, we will review and discuss the solution to each part.

-------------

## **Part 0**: Setup

In [ ]:
# Import all packages

# Use short-hand for standard packages
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt
import seaborn           as sns

# Import individual functions
from sklearn.impute          import SimpleImputer
from sklearn.model_selection import train_test_split, KFold, validation_curve, learning_curve
from sklearn.ensemble        import RandomForestRegressor, GradientBoostingRegressor
from sklearn.dummy           import DummyRegressor
from sklearn.metrics         import mean_squared_error
from sklearn.linear_model    import Ridge, Lasso
from sklearn.preprocessing   import StandardScaler
from sklearn.pipeline        import Pipeline
from sklearn.inspection      import permutation_importance
from math                    import sqrt

# Special code to ignore un-important warnings 
import warnings
warnings.filterwarnings('ignore')

# Ensure that output of plotting commands is displayed inline
%matplotlib inline 

# Set the maximum number of rows displayed 
pd.options.display.max_rows = 1000

In [ ]:
# Define all constants

SEED = 0  # base to generate a random number

# Performance metric and number of CV splits
SCORE    = 'neg_mean_squared_error'
N_SPLITS = 3

In [ ]:
pd.__version__

## **Part 1**: Data Preprocessing and EDA

First, we would like to understand the main characteristics of the dataset. We might need to transform and clean some features before we can specify a statistical model.

**Q 1:** Investigate observations with missing/null values. For each feature, what patterns can you find? 

Hint: For every feature, calculate how many observations have missing values. Also calculate the percentage of observations where each feature is missing. You can also create a heatmap using *seaborn.heatmap()* function and feed it with *dataframe.isnull()*. This approach returns an object of the same size with boolean values to indicate if values are missing. 

In [ ]:
# CODE HERE
# Load the house_data.csv file as a Pandas data frame 
data = None
print(data.shape)

# Assert OK to proceed 
assert data is not None, 'HINT: you need to complete the code to proceed.'

# Check percentage of missing data in each feature
# Count missing values and sort (descending)
total = data.isnull().sum()
total = total.sort_values(ascending=False)

# Compute percentage of missing values
percent = data.isnull().sum() / data.isnull().count()
percent = percent.sort_values(ascending=False)

# Concatenate
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(25)

**Q 2:** What's an appropriate strategy to deal with missing/null values? Implement your strategy and save a version of the dataset with no missing values in a new dataframe. 

Hint: For the features with many missing values, drop it using *dataframe.drop()*. You have to define a threshold representing "many missing values". Have a look at the number of missing values per feature computed above to find an appropriate threshold. 

In [ ]:
# Delete features with more than 200 missing values
data = data.drop((missing_data[missing_data['Total'] > 200]).index, 1)

# How did the shape change? 
data.shape

In [ ]:
# Impute missing values with most frequent value for categorical data
imp_categorical = SimpleImputer(strategy='most_frequent')

# Impute missing values with the mean for continuous data
imp_continuous  = SimpleImputer(strategy='mean')

# Impute each column with missing values
for column in data.columns:
    
    # Categorical
    if data[column].dtype == np.dtype('O'):
        data[column] = imp_categorical.fit_transform(data[column].values.reshape(-1, 1))
    
    # Continous
    else:
        data[column] = imp_continuous.fit_transform(data[column].values.reshape(-1, 1))
        
# Check number of null values in the new data frame
print(data.isnull().sum().max())

**Q 3:** After imputing missing values, skim through the description of features in the .txt file. Simply mark each feature as numeric (N), categorical (C) or ordinal (O) in the cell below.

Hint: Categorical features have no ordering semantics while ordinal features have ordering semantics.

Apply the *value_counts()* function to a data frame column to count distinct categorical/ordinal values.

In [ ]:
data['BldgType'].value_counts()

| Feature name   | Type  |
|----------------|-------|
| Id             | C     |
| MSSubClass     | C     |
| MSZoning       | C     |
| LotFrontage    | N     |
| LotArea        | N     |
| Street         | O     |
| Alley          | O     |
| LotShape       | C     |
| LandContour    | C     |
| Utilities      | O     |
| LotConfig      | C     |
| LandSlope      | C     |
| Neighborhood   | C     |
| Condition1     | C     |
| Condition2     | C     |
| BldgType       | C     |
| HouseStyle     | C     |
| OverallQual    | N     |
| OverallCond    | N     |
| YearBuilt      | N     |
| YearRemodAdd   | N     |
| RoofStyle      | C     |
| RoofMatl       | C     |
| Exterior1st    | C     |
| Exterior2nd    | C     |
| MasVnrType     | C     |
| MasVnrArea     | N     |
| ExterQual      | O     |
| ExterCond      | O     |
| Foundation     | C     |
| BsmtQual       | O     |
| BsmtCond       | O     |
| BsmtExposure   | C     |
| BsmtFinType1   | O     |
| BsmtFinSF1     | N     |
| BsmtFinType2   | O     |
| BsmtFinSF2     | N     |
| BsmtUnfSF      | N     |
| TotalBsmtSF    | N     |
| Heating        | C     |
| HeatingQC      | O     |
| CentralAir     | C     |
| Electrical     | C     |
| 1stFlrSF       | N     |
| 2ndFlrSF       | N     |
| LowQualFinSF   | N     |
| GrLivArea      | N     |
| BsmtFullBath   | N     |
| BsmtHalfBath   | N     |
| FullBath       | N     |
| HalfBath       | N     |
| BedroomAbvGr   | N     |
| KitchenAbvGr   | N     |
| KitchenQual    | O     |
| TotRmsAbvGrd   | N     |
| Functional     | O     |
| Fireplaces     | N     |
| FireplaceQu    | O     |
| GarageType     | C     |
| GarageYrBlt    | N     |
| GarageFinish   | O     |
| GarageCars     | N     |
| GarageArea     | N     |
| GarageQual     | O     |
| GarageCond     | O     |
| PavedDrive     | O     |
| WoodDeckSF     | N     |
| OpenPorchSF    | N     |
| EnclosedPorch  | N     |
| 3SsnPorch      | N     |
| ScreenPorch    | N     |
| PoolArea       | N     |
| PoolQC         | O     |
| Fence          | O     |
| MiscFeature    | C     |
| MiscVal        | N     |
| MoSold         | C     |
| YrSold         | N     |
| SaleType       | C     |
| SaleCondition  | C     |
| SalePrice      | N     |



**Q 4:** How can we numerically represent categorical and ordinal features? Perform the required feature encodings and save your pre-processed data into a new data frame. Check the data type of all features at the end and make sure they are all numeric.

Hint: Use the *get_dummies()* function in Pandas to transform categorical features into one-hot encodings. For ordinal features, try to transform them using a dictionary which maps old, non-numeric labels into new numeric labels, preserving ordering semantics. Use the *replace()* function in Pandas to transform ordinal features. Use the descriptions of features .txt file to build the dictionaries.

In [ ]:
# One hot encoding of categorical features

cols_to_transform = ['MSSubClass', 'MSZoning', 'LotShape', 'LandContour', 'LotConfig', 
                    'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
                    'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
                    'Foundation', 'Heating', 'CentralAir', 
                    'Electrical', 'MoSold', 'SaleType', 'SaleCondition', 'GarageType',
                    'BsmtExposure', 'MasVnrType']

data = pd.get_dummies(data = data, columns = cols_to_transform )

# Label encoding of ordinal features

dic_street = {'Grvl': 1, 'Pave': 2}
data.replace({'Street':dic_street},inplace=True)

dic_utilities = {'AllPub': 4, 'NoSeWa': 2}
data.replace({'Utilities':dic_utilities},inplace=True)

dic_exterqual = {'Gd':4, 'TA':3, 'Ex':5, 'Fa':2}
data.replace({'ExterQual':dic_exterqual},inplace=True)

dic_extercond = {'Gd':4, 'TA':3, 'Ex':5, 'Fa':2, 'Po':1}
data.replace({'ExterCond':dic_extercond}, inplace=True)

dic_heatingqc = {'Gd':4, 'TA':3, 'Ex':5, 'Fa':2, 'Po':1}
data.replace({'HeatingQC':dic_heatingqc}, inplace=True)

dic_kitchenqual = {'Gd':4, 'TA':3, 'Ex':5, 'Fa':2, 'Po':1}
data.replace({'KitchenQual':dic_kitchenqual}, inplace=True)

dic_functional = {'Typ':8, 'Min1':7, 'Maj1':4, 'Min2':6, 'Mod':5, 'Maj2':3, 'Sev':2}
data.replace({'Functional':dic_functional}, inplace=True)

dic_paveddrive = {'Y':3, 'N':1, 'P':2}
data.replace({'PavedDrive':dic_paveddrive}, inplace=True)

dic_garagecond = {'Gd':4, 'TA':3, 'Ex':5, 'Fa':2, 'Po':1}
data.replace({'GarageCond':dic_garagecond}, inplace=True)

dic_garagefinish = {'RFn':3, 'Unf':2, 'Fin':4, 'NA':1}
data.replace({'GarageFinish':dic_garagefinish}, inplace=True)

dic_garagequal = {'Gd':4, 'TA':3, 'Ex':5, 'Fa':2, 'Po':1}
data.replace({'GarageQual':dic_garagequal}, inplace=True)

dic_bsmtcond = {'Gd':4, 'TA':3, 'Ex':5, 'Fa':2, 'Po':1}
data.replace({'BsmtCond':dic_bsmtcond}, inplace=True)

dic_bsmtqual = {'Gd':4, 'TA':3, 'Ex':5, 'Fa':2, 'Po':1}
data.replace({'BsmtQual':dic_bsmtqual}, inplace=True)

dic_bsmtfintype1 = {'GLQ':7, 'ALQ':6, 'Unf':2, 'Rec':4, 'BLQ':5, 'LwQ':3, 'NA':1}
data.replace({'BsmtFinType1':dic_bsmtfintype1}, inplace=True)

dic_bsmtfintype2 = {'GLQ':7, 'ALQ':6, 'Unf':2, 'Rec':4, 'BLQ':5, 'LwQ':3, 'NA':1}
data.replace({'BsmtFinType2':dic_bsmtfintype2}, inplace=True)

# adjust data types to float 
data['BsmtQual'] = data['BsmtQual'].astype(float)
data['BsmtFinType1'] = data['BsmtFinType1'].astype(float)
data['BsmtFinType2'] = data['BsmtFinType2'].astype(float)
data['KitchenQual'] = data['KitchenQual'].astype(float)
data['GarageFinish'] = data['GarageFinish'].astype(float)

# Check data types of preprocessed dataframe: Are they all numeric?
# data.dtypes

**Q 5:** Why do you think the root mean square error (RMSE) between the log of predicted price and log of observed price is a better metric than a RMSE applied directly on observed and predicted prices? Visualize the distributions of the target feature and the log-transformed target feature. Also, separate your data into features and the target, where your target is the log of *SalePrice*.

Hint: Use *numpy.log1p()* to log-transform the target feature. Use *seaborn.distplot()* to visualize distributions.

In [ ]:
# Plot sales price
fig, ax = plt.subplots(figsize = (14, 8))
sns.distplot(data['SalePrice'])
plt.show()

# Plot log-transformed sales price
fig, ax = plt.subplots(figsize = (14, 8))
sns.distplot(np.log1p(data['SalePrice']))
plt.show()

In [ ]:
# Split features and target 
features = data.drop(columns = ['Id','SalePrice'])
target   = np.log1p(data['SalePrice'])

**Q 6:** Visualize the correlation of different features with the target and sort them based on the absolute value of correlation. What are the top 10 correlated values? Do they match your expectations? Why? Why not?

Hint: Compute pearson correlation of two dataframe columns A and B as follows:

*df['A'].corr(df['B'])*

In [ ]:
# Correlation of different features with respect to target

corr_dic = dict()

# Go through each column
for column in data.columns:
    
    correlation = data[column].corr(data['SalePrice'])
    corr_dic[column] = abs(correlation)

# Print top 10 correlated features
i = 0
for w in sorted(corr_dic, key=corr_dic.get, reverse=True):
    
    print('{}'.format(w).ljust(15), corr_dic[w])
    
    i = i + 1
    if (i == 10):
        break

## **Part 2**: Preparing Baseline and Cross-Validation Schema

**Q 1:** Divide your cleaned dataset into a training set and test set with ratio of 1 to 4. Dont forget to randomize your data beforehand.

Hint: Divide data into training and test sets using *train_test_split()* function in *sklearn.model_selection* package.

In [ ]:
# Separate target and features into test and training sets

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = SEED)

print('Training set: \t{}'.format(X_train.shape))
print('Test set: \t{}'.format(X_test.shape))

**Q 2:** First, compute a baseline model to compare more advanced models to later on. Consider root mean square error (RMSE) as your regression performance metric. This ensures that the error has the same unit as the target. What's the baseline performance? 

Hint: Fit an object from *DummyRegressor* class in *sklearn.dummy* package to training set and evaluate the performance on the test set. Keep the default strategy of *DummyRegressor*.

In [ ]:
# Define baseline regressor
baseline_reg = DummyRegressor()

# Fit the dummy regressor
baseline_reg.fit(X_train, y_train)

# CODE HERE
# Predict log of house prices
y_pred = None

# Assert OK to proceed 
assert y_pred is not None, 'HINT: you need to complete the code to proceed.'

# Compute root mean square error (log-transformed)
rmse_baseline = sqrt(mean_squared_error(y_test, y_pred))
print('Log-transformed RMSE: ', rmse_baseline)

## **Part 3**: Prediction using Ridge Regression

**Q 1:** Fit a ridge regression model to your data and tune the value of *alpha* parameter, the regularization strength. Note that your data should be standardized. Use a validation curve to visualize your tuning process.

Hint: Use the *Pipeline* class from *sklearn.pipeline* to make a pipeline of *StandardScaler* and *Ridge* transformers. Use the *validation_curve()* function from *sklearn.model_selection* on the training set and tune the value of *alpha*.

To visualize your tuning process, use the following function:

    def plot_validation_curve(train_scores,cv_scores,x_data,scale='lin',title='',y_label='',x_label=''):

        train_scores_mean = np.mean(train_scores, axis=1)
        train_scores_std = np.std(train_scores, axis=1)
        cv_scores_mean = np.mean(cv_scores, axis=1)
        cv_scores_std = np.std(cv_scores, axis=1)

        fig, ax = plt.subplots(figsize = (14, 8))
        plt.title(title)
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        lw = 2

        plt.fill_between(x_data, train_scores_mean - train_scores_std,train_scores_mean + train_scores_std, alpha=0.2, color="r", lw=lw)
        plt.fill_between(x_data, cv_scores_mean - cv_scores_std, cv_scores_mean + cv_scores_std, alpha=0.2, color="g", lw=lw)

        if (scale == 'lin'):
            plt.plot(x_data, train_scores_mean, 'o-', color="r", label="Training score")
            plt.plot(x_data, cv_scores_mean, 'o-', color="g",label="Cross-validation score")
        elif (scale == 'log'):
            plt.semilogx(x_data, train_scores_mean, 'o-', color="r", label="Training score")
            plt.semilogx(x_data, cv_scores_mean, 'o-', color="g",label="Cross-validation score")
        plt.grid()
        plt.legend(loc="best")
        plt.show()

In [ ]:
# Set up the pipeline
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('ridge_reg', Ridge()))
ridge_pipe = Pipeline(estimators)
ridge_pipe.set_params(ridge_reg__random_state = SEED)

# CV schema
cv_schema = KFold(n_splits = N_SPLITS, random_state = SEED)

# Tune model against a single hyper-parameter, alpha
tuning_param = 'ridge_reg__alpha'
tuning_param_range = np.logspace(-5, 3, 10)

# Tune hyper-parameter using validation curve
train_scores_val, cv_scores_val = validation_curve(
    ridge_pipe, X_train, y_train, param_name = tuning_param, param_range = tuning_param_range,
    cv = cv_schema, scoring = SCORE, n_jobs = 1)

# Define a function to facilitate drawing validation curves
def plot_validation_curve(train_scores, cv_scores, x_data, scale='lin', title='', y_label='', x_label=''):

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    cv_scores_mean = np.mean(cv_scores, axis=1)
    cv_scores_std = np.std(cv_scores, axis=1)
    
    fig, ax = plt.subplots(figsize = (14, 8))
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    lw = 2
    
    plt.fill_between(x_data, train_scores_mean - train_scores_std,train_scores_mean + train_scores_std, alpha=0.2, color="r", lw=lw)
    plt.fill_between(x_data, cv_scores_mean - cv_scores_std, cv_scores_mean + cv_scores_std, alpha=0.2, color="g", lw=lw)
    
    if (scale == 'lin'):
        plt.plot(x_data, train_scores_mean, 'o-', color="r", label="Training score")
        plt.plot(x_data, cv_scores_mean, 'o-', color="g",label="Cross-validation score")
        
    elif (scale == 'log'):
        plt.semilogx(x_data, train_scores_mean, 'o-', color="r", label="Training score")
        plt.semilogx(x_data, cv_scores_mean, 'o-', color="g",label="Cross-validation score")
        
    plt.grid()
    plt.legend(loc="best")
    plt.show()
    
# Draw validation curve
plot_validation_curve(-train_scores_val, -cv_scores_val, tuning_param_range, scale='log', 
                      title='validation curve', y_label='MSE', x_label='alpha')

# Obtain the best value of the hyper parameter
best_param_val = tuning_param_range[np.argmin(np.mean(-cv_scores_val, axis=1))]
print('Best alpha: {}'.format(best_param_val))

**Q 2:** Report the performance on test data. Is this estimation fully reliable? Why? Why not?

Hint: Set the value of *alpha* to the optimum value. Fit your pipeline to training data and predict the house prices on test data. Use *mean_squared_error()* function from *sklearn.metrics* package to measure your regression performance.

In [ ]:
# Set the optimum value for alpha
ridge_pipe.set_params(ridge_reg__alpha = best_param_val)

# Fit the ridge pipe
ridge_pipe.fit(X_train,y_train)

# Predict log of house prices
y_pred_ridge = ridge_pipe.predict(X_test)

# CODE HERE
# Compute root mean square error (log-transformed)
rmse_ridge = None

# Assert OK to proceed 
assert rmse_ridge is not None, 'HINT: you need to complete the code to proceed.'

print('Log-transformed RMSE: ', rmse_ridge)


**Q 3:** Print the list of ridge coefficients. 

Hint: Extract the ridge regression from your pipeline and use its methods directly.

In [ ]:
# Extract coefficients

ridge_reg = ridge_pipe.named_steps['ridge_reg']
ridge_coefs = ridge_reg.coef_

print('Shape: {}'.format(ridge_coefs.shape))
print(ridge_coefs)

## **Part 4**: Prediction using Lasso Regression

**Q 1:** Fit a lasso regression model to your data and tune the value of *alpha* parameter, the regularization strength. Note that your data should be standardized. Use a validation curve to visualize your tuning process.

In [ ]:
# Define pipeline
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('lasso_reg', Lasso()))
lasso_pipe = Pipeline(estimators)
lasso_pipe.set_params(lasso_reg__random_state = SEED)

# CV schema
cv_schema = KFold(n_splits = N_SPLITS, random_state = SEED)

# Tune model against a single hyper parameter
tuning_param = 'lasso_reg__alpha'
tuning_param_range = np.logspace(-5, 3, 10)

# Tune hyper-parameter using validation curve
train_scores_val, cv_scores_val = validation_curve(
    lasso_pipe, X_train, y_train, param_name = tuning_param, param_range = tuning_param_range,
    cv = cv_schema, scoring = SCORE, n_jobs = 1)

# Draw validation curve   
plot_validation_curve(-train_scores_val, -cv_scores_val, tuning_param_range, scale = 'log', 
                      title = 'validation curve', y_label = 'MSE', x_label = 'alpha')

# Obtain the best value of the hyper-parameter
best_param_val = tuning_param_range[np.argmin(np.mean(-cv_scores_val, axis=1))]
print('Best alpha: {}'.format(best_param_val))

**Q 2:** Report the performance on test data. Is this estimation fully reliable? Why? Why not?

In [ ]:
# Set the optimum value for alpha
lasso_pipe.set_params(lasso_reg__alpha = best_param_val)

# Fit the lasso pipe
lasso_pipe.fit(X_train, y_train)

# Predict log of house prices
y_pred_lasso = lasso_pipe.predict(X_test)

# Compute root mean square error (log-transformed)
rmse_lasso = sqrt(mean_squared_error(y_test, y_pred_lasso))
print('Log-transformed RMSE: ', rmse_lasso)

**Q 3:** Print the list of lasso coefficients. What's different compared to the ridge coefficients? What are the benefits and potential issues?

In [ ]:
# CODE HERE
# Extract coefficients from the pipeline
lasso_reg = None

# Assert OK to proceed 
assert lasso_reg is not None, 'HINT: you need to complete the code to proceed.'

lasso_coefs = lasso_reg.coef_

print('Shape: {}'.format(lasso_coefs.shape))
print(lasso_coefs)

## **Part 5**: Prediction using Random Forest

**Q 1:** Fit a random forest model to your data and tune the value of the *n_estimators* parameter, the number of trees in the forest. Use the validation curve to visualize your tuning process. Note that tree-based models do not require standardized features to converge well.

In [ ]:
# Define pipeline
estimators = []
estimators.append(('rf_reg', RandomForestRegressor()))
rf_pipe = Pipeline(estimators)
rf_pipe.set_params(rf_reg__random_state = SEED)

# CODE HERE
# Define the CV schema using the KFold() function in sklearn (don't forget the seed!)
cv_schema = None

# Assert OK to proceed 
assert cv_schema is not None, 'HINT: you need to complete the code to proceed.'

# Tune model against a single hyper parameter
tuning_param = 'rf_reg__n_estimators'
tuning_param_range = [int(i) for i in np.linspace(10.0, 270.0, 20)]

# Tune hyper parameter using validation curve
train_scores_val, cv_scores_val = validation_curve(
    rf_pipe, X_train, y_train, param_name = tuning_param, param_range = tuning_param_range,
    cv = cv_schema, scoring = SCORE, n_jobs = -1)

# Draw validation curve   
plot_validation_curve(-train_scores_val, -cv_scores_val, tuning_param_range, scale = 'lin', 
                      title = 'validation curve', y_label = 'MSE', x_label = 'n_estimators')

# Obtain the best value of the hyper parameter
best_param_val = tuning_param_range[np.argmin(np.mean(-cv_scores_val, axis=1))]
print('Best n_estimators: {}'.format(best_param_val))

**Q 2:** Report the performance on test data.

In [ ]:
# Set the optimum value for n_estimators
rf_pipe.set_params(rf_reg__n_estimators = best_param_val)

# Fit the lasso pipe
rf_pipe.fit(X_train, y_train)

# Predict log of house prices
y_pred_rf = rf_pipe.predict(X_test)

# Compute root mean square error (log-transformed)
rmse_rf = sqrt(mean_squared_error(y_test, y_pred_rf))
print('Log-transformed RMSE: ', rmse_rf)

**Q 3:** Print a list of the 10 most important features as defined by the decrease in mean impurity, in descending order. 

Hint: use the `feature_importances_` property of the fitted random forest model.

In [ ]:
# Extract feature importances
rf_reg = rf_pipe.named_steps['rf_reg']

i = 0
for index in reversed(np.argsort(rf_reg.feature_importances_)):
    
    print('{}'.format(features.columns[index]).ljust(20) , rf_reg.feature_importances_[index])
    
    i = i + 1
    if i == 10: break

**Q 4:** Compare the impurity-based feature importance to permutation importance on the test set. 

In [ ]:
# Compute permutation importance 
result = permutation_importance(rf_reg, X_test, y_test, n_repeats=10, random_state=SEED, n_jobs=-1)
sorted_idx = result.importances_mean.argsort()

In [ ]:
# Print top 10 most important features 
for i, col in enumerate(sorted_idx[::-1]):
    print('{}'.format(X_test.columns[col]).ljust(20), result.importances_mean[col])
    if i == 10: break

In [ ]:
# Plot permutation importance of 10 most important features 
fig, ax = plt.subplots(figsize = (14, 8))
ax.boxplot(result.importances[sorted_idx][-10:].T,
           vert=False, labels=X_test.columns[sorted_idx][-10:])
ax.set_title("Permutation Importances (TEST set)")
fig.tight_layout()
plt.show()

## **Part 6**: Prediction using Gradient Boosting Trees

**Q 1:** Fit a gradient boosting regressor tree model to your data and tune the value of the *n_estimators* parameter, the number of boosting stages to perform. Use a validation curve to visualize your tuning process.

In [ ]:
# Define pipeline
estimators = []
estimators.append(('gb_reg', GradientBoostingRegressor()))
gb_pipe = Pipeline(estimators)
gb_pipe.set_params(gb_reg__random_state = SEED)

# CV schema
cv_schema = KFold(n_splits = N_SPLITS, random_state = SEED)

# Tune model against a single hyper parameter
tuning_param = 'gb_reg__n_estimators'
tuning_param_range = [int(i) for i in np.linspace(10.0, 310.0, 25)]

# Tune hyper parameter using validation curve
train_scores_val, cv_scores_val = validation_curve(
    gb_pipe, X_train, y_train, param_name = tuning_param, param_range = tuning_param_range,
    cv = cv_schema, scoring = SCORE, n_jobs = -1)

# Draw validation curve   
plot_validation_curve(-train_scores_val, -cv_scores_val, tuning_param_range, scale = 'lin', 
                      title = 'validation curve', y_label = 'MSE', x_label = 'n_estimators')

# Obtain the best value of the hyper parameter
best_param_val = tuning_param_range[np.argmin(np.mean(-cv_scores_val, axis=1))]
print('Best n_estimators: {}'.format(best_param_val))

**Q 2:** Report the performance on test data.

In [ ]:
# Set the optimum value for n_estimators
gb_pipe.set_params(gb_reg__n_estimators = best_param_val)

# Fit the lasso pipe
gb_pipe.fit(X_train, y_train)

# Predict log of house prices
y_pred_gb = gb_pipe.predict(X_test)

# Compute root mean square error (log-transformed)
rmse_gb = sqrt(mean_squared_error(y_test, y_pred_gb))
print('Log-transformed RMSE: ', rmse_gb)

# What's the original-scale RMSE? Why do we only consider it towards the end? 
rmse = sqrt(mean_squared_error(np.exp(y_test), np.exp(y_pred_gb)))
print('Original scale RMSE:  ', rmse)

**Q 3:** Print a list of the 10 most important features as defined by the decrease in mean impurity, in descending order. What changed?

Hint: use the `feature_importances_` property of the fitted gradient boosting tree model.

In [ ]:
# Extract feature importances
gb_reg = gb_pipe.named_steps['gb_reg']

i = 0
for index in reversed(np.argsort(gb_reg.feature_importances_)):
    print('{}'.format(features.columns[index]).ljust(20) , gb_reg.feature_importances_[index])
    
    i = i + 1
    if i == 10: break

**Q 4:** Compare the impurity-based feature importance to permutation importance on the test set. What changed?

In [ ]:
# CODE HERE
# Compute permutation importance of the pipeline on test data, with 10 repeats (don't forget the seed!)
result = None

# Assert OK to proceed 
assert result is not None, 'HINT: you need to complete the code to proceed.'

sorted_idx = result.importances_mean.argsort()

In [ ]:
# Print top 10 most important features 
for i, col in enumerate(sorted_idx[::-1]):
    print('{}'.format(X_test.columns[col]).ljust(20), result.importances_mean[col])
    if i == 10: break

In [ ]:
# Plot permutation importance of 10 most important features 
fig, ax = plt.subplots(figsize = (14, 8))
ax.boxplot(result.importances[sorted_idx][-10:].T,
           vert=False, labels=X_test.columns[sorted_idx][-10:])
ax.set_title("Permutation Importances (TEST set)")
fig.tight_layout()
plt.show()

## **Part 7**: Potential Prediction Improvement with More Data

**Q 1:** A data scientist suggests that the model needs more data to improve performance. How do you know whether investing in more data indeed improves model performance? Pick the best predictive model you obtained above and visualize a learning curve, which plots the performance against training set size.

Hint: Use *learning_curve()* function from *sklearn.model_selection* to assess performance of your model using training data with different sizes. Consider 0.1, 0.2, ..., 1.0 times of the training size. Use the following function to visualize your learning curve:

    def plot_learning_curve(train_scores,cv_scores,x_data,scale='lin',title='',y_label='',x_label=''):

        train_scores_mean = np.mean(train_scores, axis=1)
        train_scores_std = np.std(train_scores, axis=1)
        cv_scores_mean = np.mean(cv_scores, axis=1)
        cv_scores_std = np.std(cv_scores, axis=1)
        
        fig, ax = plt.subplots(figsize = (14, 8))
        plt.title(title)
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        lw = 2

        plt.fill_between(x_data, train_scores_mean - train_scores_std,train_scores_mean + train_scores_std, alpha=0.2, color="r", lw=lw)
        plt.fill_between(x_data, cv_scores_mean - cv_scores_std, cv_scores_mean + cv_scores_std, alpha=0.2, color="g", lw=lw)

        if (scale == 'lin'):
            plt.plot(x_data, train_scores_mean, 'o-', color="r", label="Training score")
            plt.plot(x_data, cv_scores_mean, 'o-', color="g",label="Cross-validation score")
        elif (scale == 'log'):
            plt.semilogx(x_data, train_scores_mean, 'o-', color="r", label="Training score")
            plt.semilogx(x_data, cv_scores_mean, 'o-', color="g",label="Cross-validation score")
        plt.grid()
        plt.legend(loc="best")
        plt.show()

In [ ]:
# Pick the best predictive model
rf_reg = GradientBoostingRegressor(n_estimators = 250)
cv_schema = KFold(n_splits = N_SPLITS, random_state = SEED)

train_sizes = np.linspace(.1, 1.0, 10)

# CODE HERE
# Compute the learning curve with the learning_curve() function from sklearn
train_sizes, train_scores_learn, cv_scores_learn = None

# Assert OK to proceed 
assert train_sizes is not None, 'HINT: you need to complete the code to proceed.'

# Define a function to facilitate drawing learning curves

def plot_learning_curve(train_scores,cv_scores,x_data,scale='lin',title='',y_label='',x_label=''):
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    cv_scores_mean = np.mean(cv_scores, axis=1)
    cv_scores_std = np.std(cv_scores, axis=1)
    
    fig, ax = plt.subplots(figsize = (14, 8))
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    lw = 2
    
    plt.fill_between(x_data, train_scores_mean - train_scores_std,train_scores_mean + train_scores_std, alpha=0.2, color="r", lw=lw)
    plt.fill_between(x_data, cv_scores_mean - cv_scores_std, cv_scores_mean + cv_scores_std, alpha=0.2, color="g", lw=lw)
    
    if (scale == 'lin'):
        plt.plot(x_data, train_scores_mean, 'o-', color="r", label="Training score")
        plt.plot(x_data, cv_scores_mean, 'o-', color="g",label="Cross-validation score")
    elif (scale == 'log'):
        plt.semilogx(x_data, train_scores_mean, 'o-', color="r", label="Training score")
        plt.semilogx(x_data, cv_scores_mean, 'o-', color="g",label="Cross-validation score")
    plt.grid()
    plt.legend(loc="best")
    plt.show()
    
# Plot learning curve

plot_learning_curve(np.sqrt(-train_scores_learn), np.sqrt(-cv_scores_learn), train_sizes, scale='lin', 
              title='Learning curve of gradient boosting regressor', y_label='MSE', x_label='train set size')

**Q 2:** In general, what data-related strategies can be proposed to come up with a better prediction ?

## **Part 8**: Potential Prediction Improvement with Ensemble Model

**Q 1:** Create a simple ensemble regressor using the average of your two best models, random forest and gradient boosting regression. Compare its performance with each of the two individual models. Does performance actually improve?

In [ ]:
# CODE HERE
# Calculate average of prediction vectors (hint: use the np.mean() function)
y_pred_ensemble = None

# Assert OK to proceed 
assert y_pred_ensemble is not None, 'HINT: you need to complete the code to proceed.'

# Compute root mean square error (log-transformed)
rmse_ensemble = sqrt(mean_squared_error(y_test, y_pred_ensemble))
print('Log-transformed RMSE: ', rmse_ensemble)

rmse = sqrt(mean_squared_error(np.exp(y_test), np.exp(y_pred_ensemble)))
print('Original scale RMSE:  ', rmse)

**Q 2:** How can you make your performance results statistically more reliable?

Hint: Consider sources of selection bias in the test set.

**Q 3:** What strategies regarding modeling technique can be proposed to come up with a better prediction ?

## **SUMMARY OF RMSE VALUES**

In [ ]:
width   = 35
models  = ['Baseline', 'Ridge', 'Lasso', 'Random Forest', 'Gradient Boosting', 'Averaged Ensemble']
results = [rmse_baseline, rmse_ridge, rmse_lasso, rmse_rf, rmse_gb, rmse_ensemble]
print('', '=' * width, '\n', 'Summary of RMSE Scores'.center(width), '\n', '=' * width)  
for i in range(len(models)):
    print(models[i].center(width-8), '{0:.4f}'.format(results[i]))

## **Bonus**: Further Reading

- Intuitive guide to tree-based models: https://towardsdatascience.com/a-guide-to-decision-trees-for-machine-learning-and-data-science-fe2607241956
- Technical basics of gradient boosting: http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/
- XGBoost and why it wins Kaggle challenges: https://medium.com/syncedreview/tree-boosting-with-xgboost-why-does-xgboost-win-every-machine-learning-competition-ca8034c0b283
- More about ensemble models: https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-for-ensemble-models/
- More about permutation importance vs random forest feature importance: https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html